# LlamaIndex Api


## 初始化


## 初始化环境变量

In [ ]:
import { config } from "dotenv";

await config({ export: true });

const apiKey = Deno.env.get("API_KEY");
const baseURL = Deno.env.get("BASE_URL");
const systemPrompt = Deno.env.get("SYSTEM_PROMPT");


## 初始化 ai 实例

In [ ]:
import { OpenAI, OpenAIEmbedding } from 'llamaindex';

const llm = new OpenAI({
  apiKey: apiKey,
  model: 'gpt-4o',
  maxTokens: 4096,
  additionalSessionOptions: {
    baseURL: baseURL
  },
  temperature: 0.4,
});

const embedModel = new OpenAIEmbedding({
  apiKey: apiKey,
  model: 'text-embedding-ada-002',
  additionalSessionOptions: {
    baseURL: baseURL
  }
});

## 通过 LlamaIndex 调用 AI Api


In [ ]:
const stream = await llm.chat({
  messages: [
    {
      role: "system",
      content: systemPrompt
    },
    {
      role: "user",
      content: "生成一个登陆界面"
    }
  ],
  stream: true
})

let assistantContent = "";

for await (const chunk of stream) {
  console.log(chunk.delta);
  assistantContent += chunk.delta;
}

console.log(assistantContent);

## 快速构建一个RAG示例



In [ ]:
import { 
  Document,
  VectorStoreIndex,
  serviceContextFromDefaults,
  ContextChatEngine,
  Settings,
  CallbackManager
} from 'llamaindex';

// 创建一个文档
const document = new Document({ 
  text: assistantContent,
  id_: "login-component-doc"
});

const serviceContext = serviceContextFromDefaults({
  embedModel,
  chunkSize: 100,
  chunkOverlap: 10
});
  
// 创建一个向量存储索引
const index = await VectorStoreIndex.fromDocuments([document], { serviceContext });

// 创建一个检索器
const retriever = index.asRetriever({
  similarityTopK: 3
});

// 创建一个上下文聊天引擎
const chatEngine = new ContextChatEngine({
  retriever,
  chatModel: llm,
  systemPrompt: `根据知识回答用户的问题`
});

const createCallbackManager = () => {
  const callbackManager = new CallbackManager();
  // retrieve-end
  callbackManager.on('retrieve-end', (data) => {
    const { nodes } = data.detail;
    console.log('nodes', nodes);
    relevantContent = nodes.map((node) => ({
    content: node.node.toJSON().text,
    similarity: node.score || 0
    }));
  });
  return callbackManager;
};

createCallbackManager();

const chatFn = () => {
  return chatEngine.chat({
    message: "用到了哪些antd组件",
    stream: true
  });
};

Settings.withCallbackManager(createCallbackManager(), chatFn);

const response = await chatFn()

let responseContent = "";
for await (const chunk of response) {
  console.log(chunk?.delta);
  responseContent += chunk?.delta;
}

console.log(responseContent);
  

LlamaIndex 更多资料详见官方文档：https://ts.llamaindex.ai/docs/llamaindex